In [3]:
!pip install transformers --quiet
!pip install tokenizers --quiet
!pip install datasets --quiet
!pip install trax --quiet
!pip install wget --quiet
!pip install keras_nlp --quiet
!pip install rouge_score
# !pip install sentencepiece --quiet
# !pip install datsets transformers[sentencepiece] --quiet
import re
import keras_nlp
import json
from random import random
import math
import pandas as pd
from datasets import Dataset
import tensorflow as tf
from transformers.keras_callbacks import KerasMetricCallback
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import transformers
import datasets
from transformers import AutoTokenizer, TFT5ForConditionalGeneration, T5Tokenizer, T5Model, T5TokenizerFast,TFAutoModelForSeq2SeqLM
import datetime
import os
import wget
# import sentencepiece


In [4]:

driveLink = "https://drive.google.com/uc?export=download&id=1pX56Zk_rzTXSQWmIy08T6yJYg_fGsG6m"

data_dir = './data'
log_dir = f"{data_dir}/experiments/t5/logs"
save_path = f"{data_dir}/experiments/t5/models"
cache_path_train = f"{data_dir}/cache/t5.train"
cache_path_test = f"{data_dir}/cache/t5.test"
MODEL_NAME = "t5-base"
TRAIN_TEST_SPLIT = 0.05
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 512
BATCH_SIZE = 4
LEARNING_RATE = 3e-4
MAX_EPOCHS =10

dataAlreadyExists = os.path.exists(data_dir)
if not dataAlreadyExists:
  os.makedirs(data_dir)
  file_name = wget.download(driveLink,out = data_dir)

def wikitext_detokenizer(string):
    # contractions
    string = string.replace("s '", "s'")
    string = re.sub(r"/' [0-9]/", r"/'[0-9]/", string)
    # replace more spaces with 1
    string = re.sub("\s\s+", " ", string)
    # number separators
    string = string.replace(" @-@ ", "-")
    string = string.replace(" @,@ ", ",")
    string = string.replace(" @.@ ", ".")
    # punctuation
    string = string.replace(" : ", ": ")
    string = string.replace(" ; ", "; ")
    string = string.replace(" . ", ". ")
    string = string.replace(" ! ", "! ")
    string = string.replace(" ? ", "? ")
    string = string.replace(" , ", ", ")
    # double brackets
    string = re.sub(r"\(\s*([^\)]*?)\s*\)", r"(\1)", string)
    string = re.sub(r"\[\s*([^\]]*?)\s*\]", r"[\1]", string)
    string = re.sub(r"{\s*([^}]*?)\s*}", r"{\1}", string)
    string = re.sub(r"\"\s*([^\"]*?)\s*\"", r'"\1"', string)
    string = re.sub(r"'\s*([^']*?)\s*'", r"'\1'", string)
    # miscellaneous
    string = string.replace("= = = =", "====")
    string = string.replace("= = =", "===")
    string = string.replace("= =", "==")
    string = string.replace(" " + chr(176) + " ", chr(176))
    string = string.replace(" \n", "\n")
    string = string.replace("\n ", "\n")
    string = string.replace(" N ", " 1 ")
    string = string.replace(" 's", "'s")

    return string

def jsonToStringwithComa(entry):
    m1 = json.loads(entry)
    m2 = [list(x.values()) for x in m1]
    m3 = ''.join([item + ',' for sublist in m2 for item in sublist])
    return m3

dataPath = './data/dataDeasease.csv'
df = pd.read_csv(dataPath, delimiter=';')
df = df.dropna()
"""
question1 : what disease has these simptoms: [simptomsDesc]
Answer: [name]
"""

q1 = 'what desease has these simptoms:' + df['Sympoms desc'] + '?'
q1 = q1.str.replace('The symptoms that are highly suggestive of', '')
a1 = df['name']
"""
question2 : who Is At Risk for [name]
Answer: [whoIsAtRiskDesc]
"""
q2 = 'who Is At Risk for ' + df['name'] + '?'
a2 = df['whoIsAtRiskDesc']
"""
question3 : what are the most comon test and procedures for [name] ?
Answer: [commonTestsAndProcedures] after deJson
"""
q3 = 'what are the most comon test and procedures for ' + df['name'] + '?'
a3 = df['commonTestsAndProcedures'].map(jsonToStringwithComa)

"""
question4 : what are drugs for [name] ?
 Answer: [medications1]
"""
q4 = ' what are drugs for  ' + df['name'] + '?'
a4 = df['medications1']

q = pd.concat([q1, q2, q3, q4])
a = pd.concat([a1, a2, a3, a4])
q = q.map(wikitext_detokenizer)
a = a.map(wikitext_detokenizer)

qaPairs = pd.concat((q, a), axis=1)
qaPairs.columns = ["question", "answers_text"]
ds = tf.data.TextLineDataset(qaPairs)

dataset = Dataset.from_list(qaPairs.to_dict('records'))
raw_datasets = dataset.train_test_split(train_size=1-TRAIN_TEST_SPLIT, test_size=TRAIN_TEST_SPLIT)




In [6]:
# tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    # pad_to_max_length=True,
    inputs = [doc for doc in examples["question"]]
    # model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True,padding="max_length",add_special_tokens=True)
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)
    # the data_collator will automatically try to reduce as much as it can the size and if we use padding here it will not know to put -100 when it feeds the data
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            # examples["answers_text"], max_length=MAX_TARGET_LENGTH, truncation=True,padding="max_length",add_special_tokens=True
       examples["answers_text"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    # model_inputs["labels"] = labels["input_ids"]
    ids = np.array(labels["input_ids"])
    # to use in case DataCollator does not work ,must set the padding to labels -100 so the loss function ignores the padding
    # ids[ids == 0] = -100
    # ids= ids.tolist()
    model_inputs["labels"] = ids
    return model_inputs

# tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

#label_pad_token_id
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf",label_pad_token_id = -100,padding = 'max_length')

train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
test_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["test"]
        .select(list(range(BATCH_SIZE * 20)))
        .to_tf_dataset(
        batch_size=BATCH_SIZE,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
        )
    )
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)

rouge_l = keras_nlp.metrics.RougeL()

def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_l(decoded_labels, decoded_predictions)
    # We will print only the F1 score, you can use other aggregation metrics as well
    result = {"RougeL": result["f1_score"]}

    return result



metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

callbacks = [metric_callback]




/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Map:   0%|          | 0/2842 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
<ipython-input-6-aaf7400bac89>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ids = np.array(labels["input_ids"])


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [7]:
print(train_dataset.take(1))
list(train_dataset.take(1).as_numpy_iterator())


print(test_dataset.take(1))
list(test_dataset.take(1).as_numpy_iterator())
print(next(iter(test_dataset)))


<_TakeDataset element_spec={'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}>
<_TakeDataset element_spec={'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}>
{'input_ids': <tf.Tensor: shape=(4, 512), dtype=int64, numpy=
array([[ 125,   33, 4845, ...,    0,    0,    0],
       [ 113,   27,    7, ...,    0,    0,    0],
       [ 113,   27,    7, ...,    0,    0,    0],
       [ 113,   27,    7, ...,    0,    0,    0]])>, 'attention_mask': <tf.Tensor: shape=(4, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'labels': <tf.Tensor: shape=(4, 127), dtype=int6

In [8]:
# For now we will use our test set as our validation_data
model.fit(train_dataset, validation_data=test_dataset, epochs=MAX_EPOCHS, callbacks=callbacks)

Epoch 1/10
711/711 [==============================] - ETA: 0s - loss: 1.3677

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:838: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


711/711 [==============================] - 266s 274ms/step - loss: 1.3677 - val_loss: 0.8362 - RougeL: 0.4745
Epoch 2/10
711/711 [==============================] - 183s 258ms/step - loss: 0.8637 - val_loss: 0.6812 - RougeL: 0.4685
Epoch 3/10
711/711 [==============================] - 184s 259ms/step - loss: 0.7134 - val_loss: 0.6195 - RougeL: 0.4758
Epoch 4/10
711/711 [==============================] - 183s 257ms/step - loss: 0.6187 - val_loss: 0.5888 - RougeL: 0.4821
Epoch 5/10
711/711 [==============================] - 182s 256ms/step - loss: 0.5554 - val_loss: 0.5767 - RougeL: 0.4793
Epoch 6/10
711/711 [==============================] - 183s 257ms/step - loss: 0.5141 - val_loss: 0.5689 - RougeL: 0.4790
Epoch 7/10
711/711 [==============================] - 182s 256ms/step - loss: 0.4743 - val_loss: 0.5709 - RougeL: 0.4823
Epoch 8/10
711/711 [==============================] - 183s 257ms/step - loss: 0.4371 - val_loss: 0.5664 - RougeL: 0.4831
Epoch 9/10
711/711 [=======================

In [9]:
model.save_pretrained(save_path)
input_text =  'what desease has these simptoms:  vocal cord polyp are hoarse voice, difficulty speaking, throat swelling, and lump in throat, although you may still have vocal cord polyp without those symptoms.               ?'
encoded_query = tokenizer(input_text,
                         return_tensors='tf', pad_to_max_length=True, truncation=True, max_length=MAX_INPUT_LENGTH)
input_ids = encoded_query["input_ids"]
attention_mask = encoded_query["attention_mask"]
generated_answer = model.generate(input_ids, attention_mask=attention_mask,
                                 max_length=MAX_TARGET_LENGTH, top_p=0.95, top_k=50)
decoded_answer = tokenizer.decode(generated_answer.numpy()[0])
print("Answer: ", decoded_answer)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Answer:  <pad> Vocal cord polyp</s>


In [10]:
from google.colab import drive

drive.mount('/content/drive')



!cp -r /content/data/experiments/t5/models/ /content/drive/MyDrive/AiModels/T5HuggingFace/model2/

Mounted at /content/drive


In [12]:
def printQAPairs_vs_model_prediction_plus_rougeF1(number):
    inputText1 = qaPairs.iloc[number].values.tolist()[0]
    print("Question: ", inputText1)
    encoded_query = tokenizer(inputText1,
                              return_tensors='tf', pad_to_max_length=True, truncation=True,
                              max_length=MAX_INPUT_LENGTH)
    input_ids = encoded_query["input_ids"]
    attention_mask = encoded_query["attention_mask"]
    generated_answer = model.generate(input_ids, attention_mask=attention_mask,
                                      max_length=MAX_TARGET_LENGTH, top_p=0.95, top_k=50)
    decoded_answer = tokenizer.decode(generated_answer.numpy()[0])
    print("Answer: ", decoded_answer)
    print("Expected Answer: ", qaPairs.iloc[number].values.tolist()[1])
    resultq1 = rouge_l(qaPairs.iloc[number].values.tolist()[1], decoded_answer)
    print("Rouge: " , resultq1["f1_score"])

indexQaPairs = [18,800,1600,2400]
for i in indexQaPairs:
    printQAPairs_vs_model_prediction_plus_rougeF1(i)


Question:  what desease has these simptoms: glaucoma are symptoms of eye, spots or clouds in vision, blindness, and itchy eyelid, although you may still have glaucoma without those symptoms. ?


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Answer:  <pad> Glaucoma</s>
Expected Answer:  Glaucoma
Rouge:  tf.Tensor(0.4869021, shape=(), dtype=float32)
Question:  who Is At Risk for Spondylitis?
Answer:  <pad> Groups of people at highest risk for spondylitis include age 45-59 years. On the other hand, age 1-4 years and age <unk> 1 years almost never get spondylitis.,Within all the people who go to their doctor with spondylitis, 84% report having low back pain, 77% report having low back pain, and 77% report having leg pain.</s>
Expected Answer:  Groups of people at highest risk for spondylitis include age 45-59 years. On the other hand, age 1-4 years and age < 1 years almost never get spondylitis.,Within all the people who go to their doctor with spondylitis, 88% report having back pain, 83% report having low back pain, and 47% report having leg pain. 
Rouge:  tf.Tensor(0.48743352, shape=(), dtype=float32)
Question:  what are the most comon test and procedures for Indigestion?
Answer:  <pad> Hematologic tests (Blood test),Compl